In [1]:
from IPython.display import display, HTML

import xlrd
import pandas as pd
import geopandas
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt

import geopandas as gp

%matplotlib inline
plt.style.use('ggplot')
mlp.rcParams['figure.figsize'] = [8.0, 8.0]

In [2]:
pd.options.display.max_columns = 1000

In [3]:
basesppl = {
    2014: pd.read_excel("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/BASE DE DATOS PPL 2014.xlsx", 0, skiprows = 7),   
    2015: pd.read_excel("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/BASEDE DATOS PPL 2015.xlsx", 0, skiprows = 7),
    2016: pd.read_excel("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/BASE DE DATOS 2016.xlsx", 0, skiprows = 2),
    2017: pd.read_excel("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/BASE DE DATOS PRIVADOS DE LIBERTAD 2017.xlsx", 0, skiprows = 3)
}

In [4]:
basesppl[2017].columns=['INDEX', 'INDEX1', 'DAS', 'CENTROPENAL', 'DISTRITO', 'AGE',
       'GENDER', 'CONDITION', 'TBTYPE', 'CLASSIFICATION', 'TBLOCALIZATION',
       'DXDATE', 'DXMETHOD', 'TXDATE', 'COMORBDITIES', 'HIV']
basesppl[2016].columns=['INDEX', 'DAS', 'CENTROPENAL', 'HEALTHSERVICE', 'GENDER', 'AGE',
       'CONDITION', 'CLASSIFICATION', 'TBLOCALIZATION',
       'TXDATE', 'DXDATE', 'DXMETHOD', 'HIV', 'Unnamed: 13']
basesppl[2015].columns=['INDEX', 'AGE', 'GENDER', 'CENTROPENAL', 'MUNICIPIO', 'DEPARTAMENTO',
       'HEALTHSERVICE', 'CONDITION', 'DXDATE',
       'TXDATE', 'CLASSIFICATION', 'HIV', 'TXSCHEME']
basesppl[2014].columns=['INDEX', 'AGE', 'GENDER', 'CENTROPENAL', 'MUNICIPIO', 'DEPARTAMENTO',
       'HEALTHSERVICE', 'CONDITION', 'DXDATE',
       'TXDATE', 'CLASSIFICATION', 'HIV', 'TXSCHEME']

In [5]:
ppldfs = []
for i, b in basesppl.items(): 
    print(i, b.columns)
    b["YEAR"] = i
    ppldfs.append(b)

2014 Index(['INDEX', 'AGE', 'GENDER', 'CENTROPENAL', 'MUNICIPIO', 'DEPARTAMENTO',
       'HEALTHSERVICE', 'CONDITION', 'DXDATE', 'TXDATE', 'CLASSIFICATION',
       'HIV', 'TXSCHEME'],
      dtype='object')
2015 Index(['INDEX', 'AGE', 'GENDER', 'CENTROPENAL', 'MUNICIPIO', 'DEPARTAMENTO',
       'HEALTHSERVICE', 'CONDITION', 'DXDATE', 'TXDATE', 'CLASSIFICATION',
       'HIV', 'TXSCHEME'],
      dtype='object')
2016 Index(['INDEX', 'DAS', 'CENTROPENAL', 'HEALTHSERVICE', 'GENDER', 'AGE',
       'CONDITION', 'CLASSIFICATION', 'TBLOCALIZATION', 'TXDATE', 'DXDATE',
       'DXMETHOD', 'HIV', 'Unnamed: 13'],
      dtype='object')
2017 Index(['INDEX', 'INDEX1', 'DAS', 'CENTROPENAL', 'DISTRITO', 'AGE', 'GENDER',
       'CONDITION', 'TBTYPE', 'CLASSIFICATION', 'TBLOCALIZATION', 'DXDATE',
       'DXMETHOD', 'TXDATE', 'COMORBDITIES', 'HIV'],
      dtype='object')


In [6]:
ppldf = pd.concat(ppldfs).fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [15]:
ppldf["municode"] = ppldf.CENTROPENAL.replace({
    "GRANJA PENAL CANADA": 501, "GRANJA CANADA ": 501,
    "PREVENTIVO ZONA 18": 101,
    "CENTRO P. BARRIOS": 1801,
    "GRANJA PENAL PAVON": 113, "PAVON": 113,
    "PREVENTIVO LOS JOCOTES": 1901,
    "GRANJA PENAL CANTEL": 901,
    "ALTA SERGURIDAD, CANADA ": 501,
    "SANTA TERESA ZONA 18": 101,
    "GRANJA  PENAL PAVON": 113,
    "BOQUERON, CUILAPA": 601,
    "CENTRO FRAIJANES I": 101,
    "ALTA SEGURIDAD": 501, # El infiernito
    "CENTRO SANTA TERESA Z 18": 101, "SANTA TERESA ": 101,
    "CENTRO MAZATENANGO": 1001,
    "DELIOS MENORES ZONA 18": 101,
    "CENTRO PENAL": 1801, "CENTRO PENAL ": 1001, # Es impreciso, pero basado en la presencia de otros centros penales y en el número de casos, sospecho que es el de Puerto Barrios.
    "GRANJA PENAL CANADA, ALTA SEGURIDAD": 501,
    "SANTA TERESA PRISION": 101,
    "PREVENTIVO ": 101, "PREVENTIVO": 101, # Usualmente le llaman preventivo a los de zona 18
    "CANTEL": 901,
    "BOQUERON": 601,
    "El jocote ": 1901,
    "CENTRO PREVENTIVO": 101,
    "santa elena ": 1701,
    "ANEXO PREVENTIVO ZONA 18": 101
})

In [16]:
ppldf["CentroPenal"] = ppldf.CENTROPENAL.replace({
    "GRANJA PENAL CANADA": "GPCANADA", "GRANJA CANADA ": "GPCANADA",
    "PREVENTIVO ZONA 18": "PREVENTIVO Z18",
    "CENTRO P. BARRIOS": "CP PBARRIOS",
    "GRANJA PENAL PAVON": "PAVON", "PAVON": "PAVON",
    "PREVENTIVO LOS JOCOTES": "LOS JOCOTES",
    "GRANJA PENAL CANTEL": "CANTEL",
    "ALTA SERGURIDAD, CANADA ": "GPCANADA",
    "SANTA TERESA ZONA 18": "ST TERESA Z18",
    "GRANJA  PENAL PAVON": "PAVON",
    "BOQUERON, CUILAPA": "BOQUERON",
    "CENTRO FRAIJANES I": "FRAIJANES1",
    "ALTA SEGURIDAD": "INFIERNITO", # El infiernito
    "CENTRO SANTA TERESA Z 18": "ST TERESA Z18", "SANTA TERESA ": "ST TERESA Z18",
    "CENTRO MAZATENANGO": "MAZATE",
    "DELIOS MENORES ZONA 18": "PREVENTIVO Z18",
    "CENTRO PENAL": "CP PBARRIOS", "CP PBARRIOS": 1001, # Es impreciso, pero basado en la presencia de otros centros penales y en el número de casos, sospecho que es el de Puerto Barrios.
    "GRANJA PENAL CANADA, ALTA SEGURIDAD": "GPCANADA",
    "SANTA TERESA PRISION": "ST TERESA Z18",
    "PREVENTIVO ": "PREVENTIVO Z18", "PREVENTIVO": "PREVENTIVO Z18", # Usualmente le llaman preventivo a los de zona 18
    "CANTEL": "CANTEL",
    "BOQUERON": "BOQUERON",
    "El jocote ": "EL JOCOTE",
    "CENTRO PREVENTIVO": "PREVENTIVO Z18",
    "santa elena ": "ST ELENA",
    "ANEXO PREVENTIVO ZONA 18": "PREVENTIVO Z18"
})

In [9]:
ppldf.to_csv("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/tb-ppl-2014-2017.csv")

In [10]:
ppldf.TXDATE.head()

0    2014-07-17 00:00:00
1    2014-03-12 00:00:00
2    2014-01-21 00:00:00
3    2014-03-03 00:00:00
4    2014-03-21 00:00:00
Name: TXDATE, dtype: object

In [18]:
pplcounts = ppldf.groupby(["YEAR", "municode"]).apply(len)
pplcounts

YEAR  municode
2014  101         17
      113          5
      501         44
      601          1
      901          4
      1801        13
      1901         4
2015  101         20
      113          7
      501         49
      901          5
      1001         1
      1801        29
2016  101         39
      113         11
      501         54
      901         11
      1701         1
      1801        39
      1901         1
2017  101         41
      113         27
      501         57
      601          4
      901          6
      1001        25
      1901         3
dtype: int64

In [19]:
pplcounts.rename("n").reset_index().to_csv("../../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/PPL-conteos-2014-2017.csv")

In [20]:
print("HIV reactivity")
vih = ppldf.groupby(["HIV", "YEAR"]).INDEX.count().unstack(0, fill_value=0)
vih["total"] = vih.sum(axis=1)
np.round(vih.divide(vih.total,axis = 0)*100)

HIV reactivity


HIV,0,ND,NR,R,REHUSO,total
YEAR,,,,,,
2014,0.0,0.0,90.0,10.0,0.0,100.0
2015,0.0,0.0,95.0,5.0,0.0,100.0
2016,0.0,11.0,79.0,9.0,1.0,100.0
2017,7.0,0.0,87.0,6.0,1.0,100.0


### Diagnostic method

In [15]:
xpert = ppldf.groupby([ppldf.DXMETHOD.apply(lambda x: "xp" in str(x).lower()), 
                       "YEAR"]).INDEX.count().unstack(1, fill_value=0)#\
#    [lambda x: x.index.map(lambda x: "xp" in str(x).lower())]
#xpert.sum()
xpert.sum()

YEAR
2014     88
2015    111
2016    156
2017    163
dtype: int64

In [11]:
ppldf.head()

,AGE,CENTROPENAL,CLASSIFICATION,COMORBDITIES,CONDITION,DAS,DEPARTAMENTO,DISTRITO,DXDATE,DXMETHOD,GENDER,HEALTHSERVICE,HIV,INDEX,INDEX1,MUNICIPIO,TBLOCALIZATION,TBTYPE,TXDATE,TXSCHEME,Unnamed: 13,YEAR,municode
0,41,PREVENTIVO ZONA 18,EXTRAPULMONAR,0,NUEVO,0,GUATEMALA,0,2014-09-02 00:00:00,0,M,"CS, SANTA ELENA III",R,1.0,0.0,GUATEMALA,0,0,2014-07-17 00:00:00,C,0.0,2014,PREVENTIVO Z18
1,28,PREVENTIVO ZONA 18,BK POSITIVO,0,NUEVO,0,GUATEMALA,0,2014-03-03 00:00:00,0,M,"CS, SANTA ELENA III",NR,2.0,0.0,GUATEMALA,0,0,2014-03-12 00:00:00,A,0.0,2014,PREVENTIVO Z18
2,22,PREVENTIVO ZONA 18,EXTRAPULMONAR,0,NUEVO,0,GUATEMALA,0,2014-01-27 00:00:00,0,M,"CS, SANTA ELENA III",R,3.0,0.0,GUATEMALA,0,0,2014-01-21 00:00:00,C,0.0,2014,PREVENTIVO Z18
3,27,PREVENTIVO ZONA 18,BK POSITIVO,0,NUEVO,0,GUATEMALA,0,2014-02-28 00:00:00,0,M,"CS, SANTA ELENA III",NR,4.0,0.0,GUATEMALA,0,0,2014-03-03 00:00:00,A,0.0,2014,PREVENTIVO Z18
4,46,PREVENTIVO ZONA 18,BK POSITIVO,0,NUEVO,0,GUATEMALA,0,2014-03-21 00:00:00,0,M,"CS, SANTA ELENA III",NR,5.0,0.0,GUATEMALA,0,0,2014-03-21 00:00:00,A,0.0,2014,PREVENTIVO Z18


In [12]:
basesppl[2016].head()

,INDEX,DAS,CENTROPENAL,HEALTHSERVICE,GENDER,AGE,CONDITION,CLASSIFICATION,TBLOCALIZATION,TXDATE,DXDATE,DXMETHOD,HIV,Unnamed: 13,YEAR
0,1,ESCUINTLA,GRANJA PENAL CANADA,"CS, ESCUINTLA",M,21,NUEVO,BK POSITIVO,PULMONAR,2016-01-13,2016-01-05,BACILOSCOPIA,NR,NaN,2016
1,2,ESCUINTLA,GRANJA PENAL CANADA,"CS, ESCUINTLA",M,45,RECAIDA,BK POSITIVO,PULMONAR,2016-01-13,2016-01-05,BACILOSCOPIA,R,NaN,2016
2,3,ESCUINTLA,GRANJA PENAL CANADA,"CS, ESCUINTLA",M,21,NUEVO,BK POSITIVO,PULMONAR,2016-01-27,2016-01-20,BACILOSCOPIA,NR,NaN,2016
3,4,ESCUINTLA,GRANJA PENAL CANADA,"CS, ESCUINTLA",M,39,NUEVO,BK POSITIVO,PULMONAR,2016-01-27,2016-01-19,BACILOSCOPIA,NR,NaN,2016
4,5,ESCUINTLA,GRANJA PENAL CANADA,"CS, ESCUINTLA",M,25,NUEVO,BK POSITIVO,PULMONAR,2016-01-20,2016-01-10,BACILOSCOPIA,NR,NaN,2016


# Población PPL